In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sim.dy.model_intv import ModelIntv
import sim.dy.keys as I
import numpy.random as rd

In [2]:
seed = 11667
n_collect = 200
cvs = np.concatenate([np.linspace(0, 0.2, 11), np.linspace(0.2, 0.5, 13)[1:]])
coverage0 = 0.2

folders = [
    'dy_mul', 
    'dy_add',
    'dy_ladd'
]

In [3]:
def collect_vul(pars_post, coverages=np.linspace(0, 0.8, 33), alg='VSC'):    
    m = ModelIntv()

    mss = list()
    mss_stats = list()

    for i, pars in enumerate(pars_post):
        y1, pars = m.find_baseline(pars, 2022)

        for k in coverages:    
            _, (ms, ms_stats), _ = m.simulate_onward(y1, pars, 
                                                     intv={'FullACF': {'Coverage': k, 'ScreenAlg': alg}}, polished=True)

            ms_stats.update({'Key': i, 'Coverage': k})            
            mss_stats.append(ms_stats)
            
            ms = ms.assign(Key=i, Coverage=k)
            mss.append(ms)
            
    mss = pd.concat(mss).assign(Alg=alg)
    mss_stats = pd.DataFrame(mss_stats).assign(Alg=alg)
    return mss, mss_stats


In [ ]:
for folder in folders:
    rd.seed(seed)
    
    print(folder)
    out_path = f'out/{folder}'
    pars_post = json.load(open(f'{out_path}/Post.json', 'r'))
    
    sel = rd.choice(list(range(len(pars_post))), n_collect)
    sel.sort()
    
    pars_post = [pars_post[i] for i in sel]

    mss0, mss_stats0 = collect_vul(pars_post, alg='VSC', coverages=cvs)
    mss1, mss_stats1 = collect_vul(pars_post, alg='Sy', coverages=cvs)
    mss2, mss_stats2 = collect_vul(pars_post, alg='SyCx', coverages=cvs)

    mss = pd.concat([mss0, mss1, mss2])
    mss_stats = pd.concat([mss_stats0, mss_stats1, mss_stats2])
    mss.to_csv(f'{out_path}/Sim_VulACF_budget.csv')
    mss_stats.to_csv(f'{out_path}/Sim_VulACF_budget_stats.csv')

dy_mul
dy_add
dy_ladd
